In [ ]:
import tensorflow as tf
print(tf.__version__)

import os
os.environ['TF_DETERMINISTIC_OPS'] = '1'
os.environ['TF_CUDNN_DETERMINISTIC'] = '1'

In [ ]:
import zipfile
import os

# Specify the path to the zip file and the directory where you want to extract the contents
zip_file_path = '/content/diameterAnnotations.zip'
extracted_dir = '/content/extractions'

# Create the target directory if it doesn't exist
os.makedirs(extracted_dir, exist_ok=True)

# Open the zip file
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    # Extract all the contents to the specified directory
    zip_ref.extractall(extracted_dir)

print(f"Contents extracted to: {extracted_dir}")

Contents extracted to: /content/extractions


In [ ]:
import xml.etree.ElementTree as ET
import os
import cv2
import numpy as np
from google.colab.patches import cv2_imshow  # Import cv2_imshow for displaying images in Colab

# Function to parse the XML file and get annotations
def parse_annotations(xml_file):
    tree = ET.parse(xml_file)
    root = tree.getroot()

    annotations = []

    for image_elem in root.findall('image'):
        image_name = image_elem.get('name')
        image_annotations = []

        for polyline_elem in image_elem.findall('polyline'):
            label = polyline_elem.get('label')
            points_str = polyline_elem.get('points')
            points = [tuple(map(float, point.split(','))) for point in points_str.split(';')]

            image_annotations.append({
                'label': label,
                'points': points
            })

        annotations.append({
            'image_name': image_name,
            'annotations': image_annotations
        })

    return annotations

# Function to print annotated images
def print_annotated_images(annotations, images_dir):
    for annotation in annotations:
        image_name = annotation['image_name']
        image_path = os.path.join(images_dir, image_name)

        # Load the image using OpenCV
        image = cv2.imread(image_path)

        if image is None:
            print(f"Error loading image: {image_path}")
            continue

        for polyline in annotation['annotations']:
            label = polyline['label']
            points = np.array(polyline['points'], dtype=np.int32)  # Convert points to int32

            # Draw polyline on the image
            cv2.polylines(image, [points], isClosed=True, color=(0, 255, 0), thickness=2)

        # Display the image with annotations
        cv2_imshow(image)


xml_file_path = '/content/extractions/annotations.xml'
images_dir = '/content/images'

annotations = parse_annotations(xml_file_path)
print_annotated_images(annotations, images_dir)

In [ ]:
!pip install tensorflow opencv-python numpy yolov3-tf2
!pip install yolov3-tf2==0.3.0

segment diameters

In [ ]:
import xml.etree.ElementTree as ET
import os
import cv2
import numpy as np
from tensorflow import keras
from tensorflow.keras.optimizers import Adam  # Import Adam optimizer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

# Function to parse the XML file and get annotations
def parse_annotations(xml_file):
    tree = ET.parse(xml_file)
    root = tree.getroot()

    annotations = []

    for image_elem in root.findall('image'):
        image_name = image_elem.get('name')
        image_annotations = []

        for polyline_elem in image_elem.findall('polyline'):
            label = polyline_elem.get('label')
            points_str = polyline_elem.get('points')
            points = [tuple(map(float, point.split(','))) for point in points_str.split(';')]

            image_annotations.append({
                'label': label,
                'points': points
            })

        annotations.append({
            'image_name': image_name,
            'annotations': image_annotations
        })

    return annotations

# Load annotations
xml_file_path = '/content/extractions/annotations.xml'
images_dir = '/content/images'

annotations = parse_annotations(xml_file_path)

# Prepare data for training
X = []  # Images
labels = []  # List of Labels

for annotation in annotations:
    image_name = annotation['image_name']
    image_path = os.path.join(images_dir, image_name)

    # Load the image using OpenCV
    image = cv2.imread(image_path)

    if image is None:
        print(f"Error loading image: {image_path}")
        continue

    for polyline in annotation['annotations']:
        label_value = polyline['label']  # Use a different variable name
        points = np.array(polyline['points'], dtype=np.int32)

        # Extract the region of interest (ROI) from the image based on the points
        x, y, w, h = cv2.boundingRect(points)
        roi = image[y:y+h, x:x+w]

        # Resize the ROI to a fixed size
        roi = cv2.resize(roi, (224, 224))

        X.append(roi)
        labels.append(label_value)  # Append the label value

X = np.array(X)
labels = np.array(labels)

label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(labels)

X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42)

# Define the CNN model with Adam optimizer and a learning rate of 0.001
model = keras.Sequential([
    keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)),
    keras.layers.MaxPooling2D((2, 2)),
    keras.layers.Flatten(),
    keras.layers.Dense(128, activation='relu'),
    keras.layers.Dense(len(label_encoder.classes_), activation='softmax')  # Output layer
])

# Set the learning rate for the Adam optimizer
adam_optimizer = Adam(learning_rate=0.001)
model.compile(optimizer=adam_optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=28, validation_data=(X_test, y_test))

# Save the trained model
model.save('diameter_detection_model.h5')

# Optionally, save the label encoder for later use
import joblib
joblib.dump(label_encoder, 'label_encoder.joblib')


In [ ]:
import math

access_diameters = input("Use your ultrasound's scale to determine the length of the AP and CC diameters above. Type proceed when complete.  ").lower()

def calculate_gastric_volume():
    # Prompt the user for input

    age_input = input("Enter patient age: ")
    ap_diameter_input = input("Measure the anteroposterior diameter of the antrum and enter value: ")
    cc_diameter_input = input("Measure the craniocaudal diameter of the antrum and enter value: ")

    try:
        # Convert user inputs to float
        age = float(age_input)
        ap_diameter = float(ap_diameter_input)
        cc_diameter = float(cc_diameter_input)

        # Calculate gastric volume
        gastric_volume = 27 + (15.6 * (math.pi * ap_diameter * cc_diameter / 4)) - (1.28 * age)

        return gastric_volume
    except ValueError:
        print("Invalid input. Please enter numeric values.")
        return None

# Call the function to calculate gastric volume
if access_diameters:
  result = calculate_gastric_volume()
  if result is not None:
    print("Estimated Gastric Volume:", result)




Use your ultrasound's scale to determine the length of the AP and CC diameters above. Type proceed when complete.  proceed
Enter patient age: 23
Measure the anteroposterior diameter of the antrum and enter value: 4
Measure the craniocaudal diameter of the antrum and enter value: .6
Estimated Gastric Volume: 26.965307237600467
